# Challenge 1: Titanic - Machine Learning From Disaster 

### Khai báo thư viện

In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
import joblib
import random

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, KFold

from warnings import filterwarnings
filterwarnings('ignore')

### Tham số thực nghiệm

In [2]:
params = {}

params["exps_dir"] = "../exps"
params["exp_name"] = "challenge1_titanic_standard"

params["exps_root"] = f'{params["exps_dir"]}/result1_standard'
params["save_dir"] = f'{params["exps_dir"]}/result1_{params["exp_name"]}'

params["data_path"] = f'{params["exps_dir"]}/data/train.xlsx'

params["k_fold"] = 10
params["random_state"] = 42

random.seed(params["random_state"])
os.environ['PYTHONHASHSEED'] = str(params["random_state"])
np.random.seed(params["random_state"])

### Nạp dữ liệu

In [3]:
df_train = pd.read_excel(f"{params["data_path"]}")

In [4]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,0.271174,1,0,A/5 21171,0.014151,147,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.472229,1,0,PC 17599,0.139136,81,0
2,3,1,3,"Heikkinen, Miss. Laina",0,0.321438,0,0,STON/O2. 3101282,0.015469,147,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.434531,1,0,113803,0.103644,55,2
4,5,0,3,"Allen, Mr. William Henry",1,0.434531,0,0,373450,0.015713,147,2


### Tách X, y

In [5]:
X_train = df_train.drop("Survived", axis=1)
X_train = X_train.drop("Name",axis=1)
X_train = X_train.drop("Ticket",axis=1)
y_train = df_train["Survived"]

print(f"Train shape: {X_train.shape}, {y_train.shape}")

Train shape: (891, 9), (891,)


In [6]:
kfold = KFold(n_splits=params["k_fold"], shuffle=True, random_state=params["random_state"])
print(f"+ X_train: {len(X_train)}")
for fold, (train_idx, valid_idx) in enumerate(kfold.split(X_train, y_train)):
    print(f'Fold {fold}: ')
    print(f'+ train_idx: {train_idx}')
    print(f'+ valid_idx: {valid_idx}')
    print(f'+ train / valid: {valid_idx}')
    pass

+ X_train: 891
Fold 0: 
+ train_idx: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  24  25  26  27  28  29  31  32  33  34  35  36  37
  38  40  41  42  43  45  46  47  48  49  50  51  52  53  54  55  56  57
  58  59  60  61  62  64  65  68  69  71  73  74  75  76  77  78  79  80
  81  82  83  84  85  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 111 112 113 114 115 116 117 118
 119 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 138 139
 140 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158
 159 160 161 162 163 164 165 166 167 169 170 171 172 173 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 193 194 195 197 199
 200 201 202 203 205 206 207 209 212 213 214 216 217 218 219 220 221 222
 223 224 225 226 227 228 229 230 231 232 233 234 236 237 238 239 240 241
 242 243 245 246 247 248 249 251 252 253 255 256 257 258 259 260 261 262
 263 264 265 2

### Lựa chọn mô hình mặc định

In [7]:
models = {
    ('Logistic Regression', LogisticRegression(random_state=params["random_state"])),
    ('Decision Tree', DecisionTreeClassifier(random_state=params["random_state"])),
    ('Random Forest', RandomForestClassifier(random_state=params["random_state"])),
    ('SVM', SVC(random_state=params["random_state"])),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=params["random_state"])),
    ('XGBoost', XGBClassifier(random_state=params["random_state"]))
}

### Huấn luyện và đánh giá từng mô hình

In [8]:
results = []
baseline_results = {}

for name,model in models:
    baseline_results[name] = {"acc":[],"prec":[],"rec":[],"f1":[]}

    print(f'Model {name}:')

    kfold = KFold(n_splits=params["k_fold"], shuffle=True, random_state=params["random_state"])

    for fold, (train_idx, valid_idx) in enumerate(kfold.split(X_train, y_train)):
        X1_train, y1_train = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X1_valid, y1_valid = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        model.fit(X1_train,y1_train)

        y_pred_valid = model.predict(X1_valid)

        acc = accuracy_score(y1_valid, y_pred_valid)
        prec = precision_score(y1_valid, y_pred_valid)
        rec = recall_score(y1_valid, y_pred_valid)
        f1 = f1_score(y1_valid, y_pred_valid)

        baseline_results[name]["acc"].append(acc)
        baseline_results[name]["prec"].append(prec)
        baseline_results[name]["rec"].append(rec)
        baseline_results[name]["f1"].append(f1)
        pass

    # hiển thị tham số mô hình
    print(f'+ params = {model.get_params()}')

    # hiển thị kết quả trên từng fold
    print(f'+ acc = {baseline_results[name]["acc"]}')

    # trung bình kết quả +/ std cho tất cả các fold,
    # std: độ lệch chuẩn=> nếu cùng mean thì độ lệch nào nhỏ hơn thì tốt hơn vì ổn định và tập trung hơn
    s_msg = f'+ mean_acc = {np.mean(baseline_results[name]["acc"]):.6f} +/- {np.std(baseline_results[name]["acc"]):.6f}'
    print(s_msg)
    print()

    results.append([
        name,
        np.mean(baseline_results[name]["acc"]),
        np.mean(baseline_results[name]["prec"]),
        np.mean(baseline_results[name]["rec"]),
        np.mean(baseline_results[name]["f1"])
    ])
    #print(classification_report(y1_valid, y_pred_valid, digits=4))

Model XGBoost:
+ params = {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'feature_weights': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
+ acc = [0.8111111111111111, 0.7752808988764045, 0.

In [9]:
df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1"])
df_results = df_results.sort_values(by="F1", ascending=False)
display.display(df_results)

,Model,Accuracy,Precision,Recall,F1
4,Random Forest,0.831610,0.827296,0.713497,0.763558
1,Gradient Boosting,0.827166,0.825571,0.707284,0.757814
0,XGBoost,0.808077,0.787730,0.696843,0.734741
2,Logistic Regression,0.785618,0.737529,0.680126,0.705517
5,Decision Tree,0.756492,0.688124,0.680187,0.678079
3,SVM,0.646442,0.654684,0.183515,0.282890


Nhận xét:
+ ...